# M&M ASH benchmark Part II

This is a continuation of Part I where I use only $R=2$ conditions, 1 causal SNP, one effect size, with simple singleton, identity and fully shared patterns to ensure all computations are correct.

```
./finemap.dsc --target sanity_check -o sanity_check
```

In [1]:
%cd ~/GIT/github/mnm-twas/dsc

/home/gaow/Documents/GIT/github/mnm-twas/dsc

In [2]:
library('dscrutils')
out = dscquery('sanity_check', "hundred_data.dataset sharing_pattern.n_signal susie_scores.total susie_scores.valid susie_scores.size susie_scores.purity susie_scores.top", groups="sharing_pattern: singleton, identity, shared")

Loading dsc-query output from CSV file.
Reading DSC outputs:
 - sharing_pattern.n_signal: extracted atomic values
 - susie_scores.total: extracted atomic values
 - susie_scores.valid: extracted atomic values
 - susie_scores.size: extracted atomic values
 - susie_scores.purity: extracted atomic values
 - susie_scores.top: extracted atomic values


In [3]:
head(out)

DSC,hundred_data.dataset,sharing_pattern,sharing_pattern.n_signal,susie_scores.total,susie_scores.valid,susie_scores.size,susie_scores.purity,susie_scores.top
1,~/Documents/GTExV8/Thyroid.Lung.FMO2.filled.rds,singleton,3,1,1,27,0.7995944,0
1,~/Documents/GTExV8/Toys/Thyroid.ENSG00000031823.RDS,singleton,1,1,0,15,0.9359113,0
1,~/Documents/GTExV8/Toys/Thyroid.ENSG00000062194.RDS,singleton,2,1,1,1,1.0000000,1
1,~/Documents/GTExV8/Toys/Thyroid.ENSG00000073150.RDS,singleton,2,0,0,0,0.0000000,0
1,~/Documents/GTExV8/Toys/Thyroid.ENSG00000078319.RDS,singleton,1,1,1,64,0.8652881,0
1,~/Documents/GTExV8/Toys/Thyroid.ENSG00000081277.RDS,singleton,1,1,1,24,0.7113872,1


In [4]:
res = out[,c(3,4,5,6,7,8,9)]
colnames(res) = c('pattern', 'total_true', 'total', 'valid', 'size', 'purity', 'top_hit')

### Purity of CS

In [5]:
aggregate(purity~pattern, res, mean)

pattern,purity
identity,0.9262147
shared,0.9369584
singleton,0.7696917


### Size of CS

In [6]:
aggregate(size~pattern, res, median)

pattern,size
identity,8.0
shared,4.0
singleton,8.5


### Power

In [7]:
valid = aggregate(valid ~ pattern, res, sum)
total_true = aggregate(total_true ~ pattern, res, sum)
power = merge(valid, total_true, by = "pattern")
power$power = power$valid/power$total_true
power

pattern,valid,total_true,power
identity,119,175,0.6800000
shared,99,165,0.6000000
singleton,105,185,0.5675676


### FDR

In [8]:
valid = aggregate(valid ~ pattern, res, sum)
total = aggregate(total ~ pattern, res, sum)
fdr = merge(valid, total, by = "pattern")
fdr$fdr = (fdr$total - fdr$valid)/fdr$total
fdr

pattern,valid,total,fdr
identity,119,125,0.04800000
shared,99,106,0.06603774
singleton,105,111,0.05405405


### Top-hit rate (how often the strongest SNP is causal)

In [9]:
top_hit = aggregate(top_hit ~ pattern, res, sum)
total_true = aggregate(total_true ~ pattern, res, sum)
top_rate = merge(top_hit, total_true, by = "pattern")
top_rate$top_rate = top_rate$top_hit/top_rate$total_true
top_rate

pattern,top_hit,total_true,top_rate
identity,50,175,0.2857143
shared,55,165,0.3333333
singleton,46,185,0.2486486
